In [ ]:

import yfinance as yf
import talib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2023-01-01")


close_prices = data['Close'].values
print(f"Shape of close_prices: {close_prices.shape}")


close_prices = close_prices.flatten()
print(f"Shape of close_prices after flattening: {close_prices.shape}")


data['SMA'] = talib.SMA(close_prices, timeperiod=30)  # 30-day SMA
data['RSI'] = talib.RSI(close_prices, timeperiod=14)  # 14-day RSI
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(close_prices)  # MACD


data.dropna(inplace=True)


scaler = MinMaxScaler()
data[['SMA', 'RSI']] = scaler.fit_transform(data[['SMA', 'RSI']])


X = data[['SMA', 'RSI']]  # Features: SMA and RSI
y = data['Close']         # Target: Close price


train_size = int(len(data) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)


print("\nFirst few rows with indicators:")
print(data[['Close', 'SMA', 'RSI', 'MACD', 'MACD_signal', 'MACD_hist']].head())
print(f"\nRMSE: {rmse}")
print(f"R-squared: {r2}")